In [2]:
from io import BytesIO
from locale import currency
from os import times
from select import select
import time
from PIL import ImageGrab, Image
import win32api
import win32con
import win32gui
import ddddocr
import cv2 as cv
import numpy as np
import win32ui

In [63]:
ocr = ddddocr.DdddOcr()
name = str(input('模拟器名称：\n'))
hwnd = win32gui.FindWindow(None,name)
hwnd = win32gui.FindWindowEx(hwnd,None,None,None)
x_1, y_1, x_2, y_2 = win32gui.GetWindowRect(hwnd)
width = x_2 - x_1
height = y_2 - y_1

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
模拟器名称：
ld


In [64]:
def cvRead(img_path):
    return cv.imdecode(np.fromfile(img_path, dtype=np.uint8), -1)

In [98]:
#鼠标按下
def mouseDown(x:int, y:int):
    press_pos = win32api.MAKELONG(x,y)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, press_pos)

#鼠标松开
def mouseUp(x:int, y:int):
    press_pos = win32api.MAKELONG(x,y)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONUP,win32con.MK_LBUTTON, press_pos)

#鼠标点击
def mouseClick(x:int, y:int):
    press_pos = win32api.MAKELONG(x,y)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, press_pos)
    time.sleep(0.2)
    win32gui.PostMessage(hwnd, win32con.WM_LBUTTONUP,None, press_pos)
    
#鼠标双击
def doubleClick(x:int, y:int):
    mouseClick(x, y)
    time.sleep(0.1)
    mouseClick(x, y)

#向下滑动
def swipeDown():
    win32gui.SendMessage(hwnd, win32con.WM_KEYDOWN,win32con.VK_SPACE,0)
    time.sleep(1.2)
    win32gui.SendMessage(hwnd, win32con.WM_KEYUP,win32con.VK_SPACE,0)

def swipeLeft():
    win32gui.SendMessage(hwnd, win32con.WM_KEYDOWN,win32con.VK_CONTROL,0)
    time.sleep(1.1)
    win32gui.SendMessage(hwnd, win32con.WM_KEYUP,win32con.VK_CONTROL,0)


In [66]:
def capWindow():
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, width, height)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (width, height), dcObj, (0, 0), win32con.SRCCOPY)
    bmpinfo = dataBitMap.GetInfo()
    bmpstr = dataBitMap.GetBitmapBits(True)
    img = np.fromstring(bmpstr, dtype='uint8')
    img.shape = (height,width, 4)
    img = img[...,:3]
    img = np.ascontiguousarray(img)
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())

    return img

In [67]:
def capWindowPos(x:int, y:int, x1:int, y1:int):
    w = x1 - x
    h = y1 - y
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h), dcObj, (x,y), win32con.SRCCOPY)
    bmpinfo = dataBitMap.GetInfo()
    bmpstr = dataBitMap.GetBitmapBits(True)
    img = np.fromstring(bmpstr, dtype='uint8')
    img.shape = (h, w, 4)
    img = img[...,:3]
    img = np.ascontiguousarray(img)
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())
    
    return img

In [68]:
def OCR(x,y,x1,y1):
    w = x1 - x
    h = y1 - y
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h), dcObj, (x,y), win32con.SRCCOPY)
    bmpinfo = dataBitMap.GetInfo()
    bmpstr = dataBitMap.GetBitmapBits(True)
    img = Image.frombuffer(
        'RGB',
        (bmpinfo['bmWidth'], bmpinfo['bmHeight']),
        bmpstr, 'raw', 'BGRX', 0, 1)
    img_byte = BytesIO()  
    img.save(img_byte, format='PNG')
    img_info = img_byte.getvalue()
    result = ocr.classification(img_info)
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())

    return result

In [109]:
def clickPic(tmp:str, threshold:float):
    scr = capWindow()
    tp = cvRead(tmp)
    result = cv.matchTemplate(scr, tp, cv.TM_CCORR_NORMED)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
    if max_val < threshold:
        print(tmp, max_val, max_loc)
        return False
    #print(tmp, max_val, max_loc)
    mouseDown(int(max_loc[0]) + int(tp.shape[1]/2), int(max_loc[1]) + int(tp.shape[0]/2))
    time.sleep(0.5)
    mouseUp(int(max_loc[0]) + int(tp.shape[1]/2), int(max_loc[1]) + int(tp.shape[0]/2))
    
    return True

In [110]:
def findPic(x:int, y:int, x1:int, y1:int, tmp:str, threshold:float):
    scr = capWindowPos(x,y,x1,y1)
    tp = cvRead(tmp)
    result = cv.matchTemplate(scr, tp, cv.TM_CCORR_NORMED)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
    if max_val < threshold:
        print(tmp, max_val, max_loc)
        return False
    print(tmp, max_val, max_loc)
    return True

In [111]:
def toHome():
    while 1:
        if OCR(17,244,63,268) != '调查':
            doubleClick(54,35)
            time.sleep(2)
        else: 
            time.sleep(1)
            break

In [112]:
def addCheck():
    while 1:
        if OCR(46,258,95,287) == '图鉴':
            a = OCR(1184,208,1208,226)  
            b = OCR(1184,299,1209,318)  
            c = OCR(1217,389,1243,408)  
            d = OCR(1217,480,1243,500)  
            e = OCR(1118,665,1173,686)  
            if e == '0':
                return 0
            if (a, b, c, d).count('0') == True:
                mouseClick(1211,187)  
                time.sleep(1.5)
                mouseClick(913,643)  
                time.sleep(1)
                mouseClick(1197,283)
                time.sleep(1)
                mouseClick(913,643)  
                time.sleep(1)
                if c == '0':
                    doubleClick(1194,403)  
                    time.sleep(1)
                    doubleClick(918,636)  
                    for i in range(4):
                        doubleClick(1001,463)
                    time.sleep(1)
                    doubleClick(1020,530)
                    time.sleep(1)
                    doubleClick(895,497)
                if d == '0':
                    doubleClick(1197,522)  
                    time.sleep(1)
                    doubleClick(918,636)  
                    for i in range(4):
                        doubleClick(1001,463)
                    time.sleep(1)
                    doubleClick(1020,530)
                    time.sleep(1)
                    doubleClick(895,497)
                doubleClick(54,35)
            if (a,b,c,d).count('0') == False:
                break

In [113]:
def pull():
    addCheck()
    time.sleep(1)
    doubleClick(1156,592)
    time.sleep(1.3)
    doubleClick(1156,592)

In [114]:
def drag():
    while 1:
        if findPic(466,109,634,151,'钓鱼\\蓝色.bmp',0.98):
            mouseDown(1156,592)
        elif findPic(740,98,811,170,'钓鱼\\绿色.bmp',0.99):
            mouseDown(1156,592)
        elif findPic(901,155,941,229,'钓鱼\\红色.bmp',0.97):
            mouseUp(1156,592)   
        if OCR(572,639,704,671) == '点击屏幕继续':
            time.sleep(0.5)
            doubleClick(639,651)
            time.sleep(3)
            doubleClick(639,651)
            time.sleep(3)
            doubleClick(948,605)  
            if OCR(543,127,602,161) == '来自':
                time.sleep(2)
                doubleClick(663,348)
                time.sleep(4)
                doubleClick(658,589)
            break
        if OCR(581,159,752,209) == '鱼儿跑了':
            time.sleep(2)
            break

In [115]:
def join(point):
    toHome()
    time.sleep(1.5)
    doubleClick(736,665)
    time.sleep(2)
    points = [
        '钓鱼\\1汇湾海滨.bmp',
        '钓鱼\\2湖湾公园.bmp',
        '钓鱼\\3白金海岸.bmp',
        '钓鱼\\4牧森湖.bmp',
        '钓鱼\\5.斯皮亚水库.bmp',
        '钓鱼\\6云霞群岛.bmp'
    ]
    result = findPic(480,48,1260,686,points[point],0.98)
    if result:
        clickPic(points[point],0.99)
    else:
        time.sleep(2)
        swipeDown()
        time.sleep(4)
        clickPic(points[point],0.99)

In [116]:
def fishing(times:int,point:int):
    _times = 0
    join(point)
    time.sleep(2)
    while _times < times:
        pull()
        time.sleep(2)
        drag()
        _times += 1

In [ ]:
#a为钓鱼次数，b为钓鱼点====>0表示湖湾公园，5表示云霞群岛，只能选择0-5
"""修改a和b的值，然后运行"""
a = 5
b = 5

fishing(a, b)